In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import bisect
from ipywidgets import interact, FloatSlider, IntSlider

def catenary(x, a):
    """Returns the y-coordinate of a standard catenary at position x with parameter a."""
    return a * np.cosh(x / a)

def arc_length(a, x0, x1):
    """
    Calculates the arc length between x0 and x1 of a catenary with parameter a.
    Uses the analytical formula for the arc length of a catenary.
    """
    return a * (np.sinh(x1 / a) - np.sinh(x0 / a))

def find_x_for_arc_length(a, x_start, target_length):
    """
    Finds the value of x such that the arc length from x_start to x equals target_length.
    Uses the bisection method to numerically solve the equation.
    """
    func = lambda x: arc_length(a, x_start, x) - target_length
    x_upper = x_start + target_length * 2  # Reasonable upper bound
    return bisect(func, x_start, x_upper)

def plot_equal_arc_segments(a=3.0, n_segments=16, span=20.0):
    """
    Plots an inverted catenary divided into equal arc-length segments.
    
    Parameters:
    - a: catenary parameter.
    - n_segments: number of segments to divide the curve into.
    - span: total horizontal width of the catenary.
    """
    # Define the horizontal domain of the catenary
    x_min = -span / 2
    x_max = span / 2

    # Compute total arc length of the catenary
    total_length = arc_length(a, x_min, x_max)
    
    # Calculate the target length for each segment
    segment_length = total_length / n_segments

    # Compute x-coordinates dividing the arc into equal-length segments
    x_points = [x_min]
    for i in range(1, n_segments):
        # Find the next x such that the arc from the last x equals segment_length
        x_next = find_x_for_arc_length(a, x_points[-1], segment_length)
        x_points.append(x_next)
    x_points.append(x_max)  # Ensure the last point aligns with x_max

    x_points = np.array(x_points)
    y_points = -catenary(x_points, a)  # Inverted catenary

    # Generate smooth curve for the full catenary
    x_curve = np.linspace(x_min, x_max, 1000)
    y_curve = -catenary(x_curve, a)

    # Plot the catenary and its segments
    plt.figure(figsize=(12, 6))
    plt.plot(x_curve, y_curve, label='Inverted Catenary', color='orange', lw=2)

    segment_lengths = []
    for i in range(n_segments):
        x1, y1 = x_points[i], y_points[i]
        x2, y2 = x_points[i + 1], y_points[i + 1]
        
        # Compute and plot each segment with its length
        length = np.hypot(x2 - x1, y2 - y1)
        segment_lengths.append(length)
        plt.plot([x1, x2], [y1, y2], color='saddlebrown', lw=3)
        plt.text((x1 + x2) / 2, (y1 + y2) / 2, f'{length:.2f} m', fontsize=8, ha='center', color='blue')

    # Mark connection points between segments
    plt.plot(x_points, y_points, 'o', color='navy', label='Connection Points')
    plt.title(f'Segmented Inverted Catenary | a = {a}, Segments = {n_segments}')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.axis('equal')
    plt.grid(True)
    plt.legend()

    # Display total length and standard deviation
    total_segment_length = sum(segment_lengths)
    std_dev = np.std(segment_lengths)
    plt.figtext(0.15, 0.88, f'Total Length: {total_segment_length:.2f} m', fontsize=11, color='green')
    plt.figtext(0.15, 0.83, f'Segment Length Std Dev: {std_dev:.4f} m', fontsize=10, color='red')
    plt.show()

    # Plot bar chart of individual segment lengths
    plt.figure(figsize=(10, 3))
    plt.bar(range(1, n_segments + 1), segment_lengths, color='skyblue')
    plt.title('Segment Lengths')
    plt.xlabel('Segment Index')
    plt.ylabel('Length (m)')
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.show()

# Interactive controls for parameter adjustment in a Jupyter Notebook
interact(
    plot_equal_arc_segments,
    a=FloatSlider(min=0.5, max=15.0, step=0.1, value=5.0, description='Parameter a'),
    n_segments=IntSlider(min=2, max=32, step=1, value=8, description='Segments'),
    span=FloatSlider(min=5.0, max=40.0, step=1.0, value=20.0, description='Width (m)')
)


interactive(children=(FloatSlider(value=5.0, description='Parameter a', max=15.0, min=0.5), IntSlider(value=8,…

<function __main__.plot_equal_arc_segments(a=3.0, n_segments=16, span=20.0)>